# Ejecutable Red Convolucional

## Imports

In [13]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

## Dataset

In [ ]:
# Configuramos las rutas de los directorios de las imágenes que vamos a utilizar para la red
path = "path"

# Configuramos el tamaño de las imágenes y el batch_size
img_size = (225, 225)
batch_size = 48

# Declaramos las variables donde vamos a guardar los resultados
img_names = []
img_predictions = []

# Creamos los datasets de entrenamiento y validación y los configuramos
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1.0 / 255.0, validation_split = 0.2)

train_generator = datagen.flow_from_directory(
    path,
    target_size = img_size,
    batch_size = batch_size,
    class_mode = "sparse",
    color_mode = "rgb",
    subset = "training"
)

validation_generator = datagen.flow_from_directory(
    path,
    target_size = img_size,
    batch_size = batch_size,
    class_mode = "sparse",
    color_mode = "rgb",
    subset = "validation"
)

# Guardamos los nombres de cada clase para luego asignarlos
class_indices = train_generator.class_indices

index_to_class = {v: k for k, v in class_indices.items()}

print("Set de entrenamiento: ", len(train_generator))
print("Set de validacion: ", len(validation_generator))

plt.imshow(train_generator[0][0][0])

plt.show()

## Modelo

### Estructura

In [15]:
def create_model():
    input = tf.keras.layers.Input(shape=(img_size[0], img_size[1], 3))

    x = tf.keras.layers.Conv2D(32, (3, 3), activation = "relu")(input)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)

    x = tf.keras.layers.Conv2D(64, (3, 3), activation = "relu")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Dropout(0.3)(x)

    x = tf.keras.layers.Conv2D(128, (3, 3), activation = "relu")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Dropout(0.3)(x)

    x = tf.keras.layers.Conv2D(256, (3, 3), activation = "relu")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Dropout(0.3)(x)

    x = tf.keras.layers.Conv2D(512, (3, 3), activation = "relu")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Dropout(0.4)(x)

    x = tf.keras.layers.Flatten()(x)

    x = tf.keras.layers.Dense(512, activation = "relu")(x)
    x = tf.keras.layers.Dropout(0.4)(x)

    output = tf.keras.layers.Dense(len(class_indices), activation='softmax')(x)

    return tf.keras.models.Model(input, output)

### Carga de los Pesos al Modelo

In [ ]:
weight_path = "weight_path"

model = create_model()
model.summary()

model.load_weights(weight_path)

# Prueba Modelo

### Predicción de Imágenes

In [ ]:
img_prueba_path = "img_prueba_path"

for image in os.listdir(img_prueba_path):
    img = cv2.imread(os.path.join(img_prueba_path, image), cv2.IMREAD_COLOR)
    resized_img = cv2.resize(img, (225, 225), interpolation=cv2.INTER_AREA)
    imgRGB = cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB)
    imgRGB_batch = np.expand_dims(imgRGB, axis=0)
    prediction = model.predict(imgRGB_batch)
    predicted_class = np.argmax(prediction, axis=1)[0]
    predicted_class_name = index_to_class[predicted_class]
    
    print(predicted_class_name)
    
    plt.plot()
    plt.imshow(imgRGB)
    plt.show()


### Reconocimiento a tiempo real

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocesar el frame para el modelo
    resized_frame = cv2.resize(frame, (225, 225), interpolation=cv2.INTER_AREA)
    frame_rgb = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
    frame_rgb_batch = np.expand_dims(frame_rgb, axis=0)

    # Realizar predicción
    prediction = model.predict(frame_rgb_batch)
    predicted_class = np.argmax(prediction, axis=1)[0]
    predicted_class_name = index_to_class[predicted_class]

    # Mostrar la predicción en el frame
    cv2.putText(frame, f"Predicción: {predicted_class_name}", 
                (10, 50),  # Coordenadas del texto
                cv2.FONT_HERSHEY_SIMPLEX,  # Fuente
                1,  # Escala de la fuente
                (0, 255, 0),  # Color (verde en este caso)
                2,  # Grosor de la línea
                cv2.LINE_AA)  # Tipo de línea

    # Mostrar el frame con texto
    cv2.imshow('Objeto', frame)

    # Salir si se presiona la tecla ESC
    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()